In [65]:
import numpy as np

## 折叠共源共栅放大器

电路图如下

<img src="https://xkraft-blog-1211-1314103412.cos.ap-nanjing.myqcloud.com/img/%E7%94%B5%E8%B7%AF%E5%9B%BE.jpg" alt="电路图" style="zoom: 25%;" />

#==========================================================#内是需要手动设置的参数

以下计算均取国际标准单位

In [66]:
#==========================================================#
#性能指标
VDD = 3.3 #单位V
GB = 3 * 10**(6) #增益带宽，单位：Hz
SR = 5 * 10**(6) #摆率，单位：V/s
ICMR_MIN = 1.1 #输入共模范围最小值，单位：V
ICMR_MAX = 2.5 #输入共模范围最大值，单位：V
phase_margin = 50 #相位裕量，单位：°
Vout_MIN = 0.6 #- 0.1 #输出电压最小值，单位V
Vout_MAX = 2.7 #+ 0.1 #输出电压最大值，单位V
Pdiss = 5 * 10**(-3) #功耗，单位W
#Av = 3000 #增益
CL = 10 * 10**(-12) #负载电容，单位F，暂时定为10pF
lemdaP = 0.05
lemdaN = 0.04
#==========================================================#

In [67]:
s_si = 3.9 * 8.85 * 10**(-12) #硅的介电常数，单位F/m

#==========================================================#
tnox = 6.65 * 10**(-9) #NMOS栅氧厚度，单位m
uno = 35 * 10**(-3) #NMOS沟道迁移率，单位m^2/V/s
Vthn = 0.695 #NMOS阈值电压，单位V

tpox = 6.62 * 10**(-9) #PMOS栅氧厚度，单位m
upo = 9.25 * 10**(-3) #PMOS沟道迁移率，单位m^2/V/s
Vthp = -0.672 #PMOS阈值电压，单位V
#==========================================================#

Kn = uno * s_si / tnox
Kp = upo * s_si / tpox
print('Kn:{:.6e}'.format(Kn))
print('Kp:{:.6e}'.format(Kp))

Kn:1.816579e-04
Kp:4.822715e-05


In [68]:
#米勒补偿电容Cc > 0.22CL
Cc = 10 * 10**(-12) #单位F

S = np.zeros((15,), dtype=float)#S[0]不用
I = np.zeros((15,), dtype=float)#I[0]不用
uo = np.zeros((15,), dtype=float)#uo[0]不用

uo[14] = uo[4] = uo[5] = uo[13] = uo[6] = uo[7] = upo
uo[1] = uo[2] = uo[3] = uo[8] = uo[9] = uo[10] = uo[11] = uo[12] = uno
#确定尾电流I3
I[3] = SR * Cc * 1
print('I3:{:.6e}'.format(I[3]))

I3:5.000000e-05


In [69]:
#==========================================================#
#设置一个合适的比例，需在1.2到1.5之间
rate1 = 1.4
#==========================================================#
#输出共源共栅的偏置电流
I[4] = I[5] = rate1 * I[3]
print('I4、I5:{:.6e}'.format(I[4]))

I4、I5:7.000000e-05


In [70]:
Vsd5 = Vsd7 = (VDD - Vout_MAX) / 2
print('Vsd5 = Vsd7 :{:.6e}'.format(Vsd5))
S[4] = S[5] = S[14] = (2 * I[5]) / (Kp * Vsd5**(2))
print('S4、S5、S14:{:.6e}'.format(S[4]))

Vsd5 = Vsd7 :3.000000e-01
S4、S5、S14:3.225477e+01


In [71]:
#假设M6、M7电流处于最坏情况
I[6] = I[7] = I[5]
S[6] = S[7] = S[13] = (2 * I[7]) / (Kp * Vsd7**(2))
print('S6、S7、S13:{:.6e}'.format(S[6]))

S6、S7、S13:3.225477e+01


In [72]:
Vsd9 = Vsd11 = (Vout_MIN - 0) / 2
print('Vsd9 :{:.6e}'.format(Vsd9))
I[8] = I[9] = I[10] = I[11] = I[6]
S[8] = S[9] = S[10] = S[11] = (2 * I[9]) / (Kn * Vsd9**(2))
print('S8、S9、S10、S11:{:.6e}'.format(S[8]))

Vsd9 :3.000000e-01
S8、S9、S10、S11:8.563105e+00


In [73]:
S[1] = S[2] = ((2 * np.pi * GB)**(2) * CL**(2)) / (Kn * I[3])
print('S1、S2:{:.6e}'.format(S[1]))

S1、S2:3.911812e+00


In [74]:
#S[3] = (2 * 5.000000e-05) / (Kn * ( (ICMR_MIN - np.sqrt(5.000000e-05 / (Kn * 3.9)) - Vthn))**(2))
S[3] = (2 * I[3]) / (Kn * ( (ICMR_MIN - np.sqrt(I[3] / (Kn * S[1])) - Vthn))**(2))
vds3 = ((ICMR_MIN - np.sqrt(I[3] / (Kn * S[1])) - Vthn))
print('vds3:{:.6e} V'.format(vds3))
print('S3:{:.6e}'.format(S[3]))

vds3:1.397418e-01 V
S3:2.818988e+01


In [75]:
#验证S4、S5是否足够大以满足最大输入共模电压
S_temp = (2 * I[4]) / (Kp * (VDD - ICMR_MIN - Vthp)**(2))
print('S_temp:{:.6e}'.format(S_temp))
if(S[3] >= S_temp):
    print("满足条件")
else:
    print("不满足条件，需重新设计S3、S4")

S_temp:3.519391e-01
满足条件


In [76]:
S[12] = (S[3] * I[4]) / I[3]
print('S12:{:.6e}'.format(S[12]))

S12:3.946584e+01


In [77]:
#功耗计算
P = VDD * (I[4] * 3)
print('P:{:.6e}'.format(P))
if(P < Pdiss):
    print("功耗满足要求")
else:
    print("功耗不满足要求，请重新设计")

P:6.930000e-04
功耗满足要求


In [78]:
#计算小信号模型下的跨导
g = np.zeros((15,), dtype=float)#g[0]不用
print('I4、I5:{:.6e} A'.format(I[4]))
#Kp、Kn的单位是A/V^2 I的单位是A S标量 开根量纲是A/V 也就是S
g[4] = g[5] = g[13] = g[14] = np.sqrt(2 * I[5] * Kp * S[5])
g[6] = g[7] = np.sqrt(2 * (I[5] - I[3] / 2) * Kp * S[7])
g[8] = g[9] = g[10] = g[11] = np.sqrt(2 * (I[5] - I[3] / 2) * Kn * S[11])
g[1] = g[2] = np.sqrt(2 * (I[3] / 2) * Kn * S[1])
print('g[4]、g[5]、g[13]、g[14]：{:.6e} S'.format(g[4]))
print('g[6]、g[7]：{:.6e} S'.format(g[6]))
print('g[8]、g[9]：{:.6e} S'.format(g[8]))
print('g[1]、g[2]：{:.6e} S'.format(g[1]))
gds = np.zeros((15,), dtype=float)#gds[0]不用
gds[4] = gds[5] = gds[13] = gds[14] = I[5] * lemdaP
gds[6] = gds[7] = (I[5] - I[3] / 2)  * lemdaP
gds[8] = gds[9] = gds[10] = gds[11] = (I[5] - I[3] / 2) * lemdaN
gds[1] = gds[2] = (I[3] / 2) * lemdaN
print('gds[4]、gds[5]、gds[13]、gds[14]：{:.6e} S'.format(gds[4]))
print('gds[6]、gds[7]：{:.6e} S'.format(gds[6]))
print('gds[8]、gds[9]、gds[10]、gds[11]：{:.6e} S'.format(gds[8]))
print('gds[1]、gds[2]：{:.6e} S'.format(gds[1]))

I4、I5:7.000000e-05 A
g[4]、g[5]、g[13]、g[14]：4.666667e-04 S
g[6]、g[7]：3.741657e-04 S
g[8]、g[9]：3.741657e-04 S
g[1]、g[2]：1.884956e-04 S
gds[4]、gds[5]、gds[13]、gds[14]：3.500000e-06 S
gds[6]、gds[7]：2.250000e-06 S
gds[8]、gds[9]、gds[10]、gds[11]：1.800000e-06 S
gds[1]、gds[2]：1.000000e-06 S


In [79]:
#计算R9、R11、k
rds = np.zeros((15,), dtype=float)#rds[0]不用
R = np.zeros((15,), dtype=float)#R[0]不用

rds[4] = rds[5] = rds[13] = rds[14] = 1/gds[4]
rds[6] = rds[7] = 1/gds[6]
rds[8] = rds[9] = rds[10] = rds[11] = 1/gds[8]
rds[1] = rds[2] = 1/gds[1]

print('rds[4]、rds[5]、rds[13]、rds[14]：{:.6e} Ω'.format(rds[4]))
print('rds[6]、rds[7]：{:.6e} Ω'.format(rds[6]))
print('rds[8]、rds[9]、rds[10]、rds[11]：{:.6e} Ω'.format(rds[8]))
print('rds[1]、rds[2]：{:.6e} Ω'.format(rds[1]))

R[9] = g[9] * rds[9] * rds[11]
#r[1]= g[6] * rds[6] * (rds[1] * rds[4]/(rds[1] + rds[4]))
r = g[6] * rds[6] * (1/(gds[1] + gds[4]))
R[11] = (R[9] * r)/(R[9] + r)
k =R[9] * (gds[2] + gds[4])/(g[7] * rds[7]) #k
Avd = (2 + k)/(2 + 2 * k) * g[1] * R[11] #Avd

print('r：{:.6e} Ω'.format(r))
print('R[9]：{:.6e} Ω'.format(R[9]))
print('R[11]：{:.6e} Ω'.format(R[11]))
print('k:{:.6e}'.format(k))
print('Avd:{:.6e}'.format(Avd))

rds[4]、rds[5]、rds[13]、rds[14]：2.857143e+05 Ω
rds[6]、rds[7]：4.444444e+05 Ω
rds[8]、rds[9]、rds[10]、rds[11]：5.555556e+05 Ω
rds[1]、rds[2]：1.000000e+06 Ω
r：3.695464e+07 Ω
R[9]：1.154833e+08 Ω
R[11]：2.799594e+07 Ω
k:3.125000e+00
Avd:3.278205e+03


In [80]:
#电流漏的设计计算,取最小的作为偏置电压
Vbias = np.zeros((15,), dtype=float)#Vbias[0]不用
#Slou = np.zeros((15,), dtype=float)#Slou[0]不用
#Von = np.zeros((15,), dtype=float)#Slou[0]不用
Rref = np.zeros((15,), dtype=float)#Slou[0]不用

Slou = 8.8
Vbias[3] = np.sqrt(2 * I[3] / (Kn * S[3])) + Vthn
Vbias[12] = np.sqrt(2 * I[4] / (Kn * S[12])) + Vthn
print('Vbias[3]：{:.6e} V'.format(Vbias[3]))
print('Vbias[12]：{:.6e} V'.format(Vbias[12]))
#Von is vgs- vthn,so vgs1 is vbias3
Von = Vbias[3] - Vthn
print('Von：{:.6e} V'.format(Von))
#Slou[1] = Slou[2] = Slou[3] = Slou[5] = 2 * I[3] / (Kn * Von[1] * Von[1])
#Slou = 2 * I[3] / (Kn * Von * Von)

print('Slou[1]、Slou[2]、Slou[3]、Slou[5]：{:.6e} '.format(Slou))
Rref[1] = (3.3 - Vthn - 2*Von)/I[3]
Rref[2] = (3.3 - Vthn - Von)/I[3]
print('Rref[1]：{:.6e} Ω'.format(Rref[1]))
print('Rref[2]：{:.6e} Ω'.format(Rref[2]))
print('S3:{:.6e}'.format(S[3]))



Vbias[3]：8.347418e-01 V
Vbias[12]：8.347418e-01 V
Von：1.397418e-01 V
Slou[1]、Slou[2]、Slou[3]、Slou[5]：8.800000e+00 
Rref[1]：4.651033e+04 Ω
Rref[2]：4.930516e+04 Ω
S3:2.818988e+01
